In [2]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from math import cos,sin
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact
import plotly.express as px
import pickle

import ipywidgets
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox, Output, HTML, Dropdown, Button, Layout, Label
from IPython.display import display, clear_output

import requests
import json

import gc
from scipy.stats import norm

from sklearn.preprocessing import LabelBinarizer

In [49]:
class TemperatureApp:
    
    """
    Framework for an IPython notebook todo list which allows
    additions and deletions to a sorted list of things to do.
    """
    
    def __init__(self):
        "build the widgets and assemble them into a display."
        with open("global_df.pkl","rb") as f:
            self.global_df = pickle.load(f)
        with open("global_ridge_model.pkl","rb") as f:
            self.global_ridge_lm = pickle.load(f)
        with open("country_yearly_df.pkl","rb") as f:
            self.country_yearly_df = pickle.load(f)
        with open("country_yearly_nn_model.pkl","rb") as f:
            self.country_yearly_nn = pickle.load(f)
        with open("country_monthly_df.pkl","rb") as f:
            self.country_monthly_df = pickle.load(f)
        with open("country_monthly_nn_model.pkl","rb") as f:
            self.country_monthly_nn = pickle.load(f)
        with open("city_monthly_sample.pkl","rb") as f:
            self.city_monthly_df = pickle.load(f).query("Year < 2058")
        with open("city_monthly_nn_model.pkl","rb") as f:
            self.city_monthly_nn = pickle.load(f)
        
        
        
        self.global_ridge_lm_features = ['Gas consumption','Population','Coal consumption','Oil consumption','Gas cumsum', \
                                         'Coal cumsum','Oil cumsum','FossilFuelGrowth','CoalGrowth','GasGrowth','OilGrowth']
        
        self.city_monthly_nn_columns = ['Year','x','y','z','TempMinus1','TempMinus2', \
             'January','February','March','April','May','June','July','August','September','October','November','December']
  
        style="""
            <style>
                /* enlarges the default jupyter cell outputs, can revert by Cell->Current Outputs->Clear */
                .container { width:1020 !important; } 
                
                /* styles for output widgets */
                .o2 {width:400px; border:1px solid #ddd}
                .o3 {width:400px; border:1px solid #ddd}
                .o4 {width:400px; border:1px solid #ddd}
                .o5 {width:800px; }
                .o5 span {color:red !important}
                
                /* custom styles for testing */
                .style_A {background-color:#fafaaa}
                .style_B {background-color:#faaafa}
                .style_C {background-color:#aafafa}
            </style>
        """
        display(HTML(style))
        self.o1 = Output(layout=Layout(width='400px'))
        self.o2 = Output() 
        self.o2.add_class('o2')
        self.o3 = Output()
        self.o3.add_class('o3')
        self.o4 = Output()
        self.o4.add_class('o4')
        self.o5 = Output()
        self.o5.add_class('o5')
        # create a scene for displaying the outputs, 
        # Output1 on the top row, 2,3, and 4 stacked horizontally in the second row
        scene = VBox([self.o1,
                      HBox([self.o2, self.o3, self.o4]),
                      self.o5
                     ])
        display(scene)
    
        with self.o1:
            display(HTML('<h2>Team Cowboys</h2>'))
        with self.o2:
            self.dd_filter = Dropdown(description='Select Scale', options=['', 'Global Average Temperatures', \
                           'Country Average Temperatures','Personalized Report'])
            layout = Layout(width='100px',height='30px')
            self.lat_w = widgets.BoundedFloatText(value=0,min=-180,max=180,layout=layout)
            self.lon_w = widgets.BoundedFloatText(value=0,min=-180,max=180,layout=layout)
            
            self.lat_box = VBox([Label('Latitude'),self.lat_w])
            self.lon_box = VBox([Label('Longitude'),self.lon_w])
            self.coords = HBox([self.lat_box,self.lon_box])

            self.btn = Button(description='Run')
            
            self.btn.on_click(self.display_results)
            
            
            display(self.dd_filter, self.coords,self.btn)
            lbl = Label(value=f'Dallas: 32.95, -96.70.')
            display(lbl)
            lbl = Label(value=f'Louisville: 37.78, -85.42.')
            display(lbl)
            lbl = Label(value=f'Washington: 39.38, -76.99.')
            display(lbl)
            
            
    def display_results(self, x):
            
        filter_by = self.dd_filter.value


        if filter_by=='':
            with self.o3:
                clear_output()
                print('Please select scale.')
            return

        if filter_by == 'Global Average Temperatures':
            df = self.global_df
            lm = self.global_ridge_lm

        elif filter_by == 'Country Average Temperatures':
            df = self.country_yearly_df
            lm = self.country_yearly_nn

        else:
            df = self.city_monthly_df
            lm = self.city_monthly_nn
        
        
        with self.o3:
            clear_output()
            lon = self.lon_w.value
            lat = self.lat_w.value
            api_key = "d7f6fcc66c3bd5ab43f7e2b871cf089a"
            url = "https://api.openweathermap.org/data/2.5/weather?lat="+str(lat)+"&lon="+str(lon)+"&appid=" + str(api_key) + "&units=metric"
            
            current_temp = json.loads(requests.request("GET",url).text)['main']['temp']
            current_month = dt.datetime.now().strftime('%m').lstrip('0')
            current_month_text = dt.datetime.now().strftime('%B')
            current_year = dt.datetime.now().strftime('%Y')
            
            mydf = df.query("Latitude == "+str(lat)+" and Longitude =="+str(lon))
            
            current_city = mydf.iloc[0]['City']
            
            pred = float(df.query("Latitude == "+str(lat)).query("Year == "+str(current_year)+" and Month == "+str(current_month))['AverageTemperature'])
             
            lbl = Label(value=f'City: {current_city}. Month: {current_month_text}. Temp: {current_temp} Celsius.')
            lbl.add_class(f'style_{filter_by}')
            
            display(lbl)
    
            
            
            
            x = np.arange(pred-15, pred+15, 0.1)
            plt.plot(x, norm.pdf(x, pred, 1.55))
            plt.axvline(x=current_temp, color='red', linestyle='--')
            plt.show()
            
            if current_temp - pred > 0:
                direction = "higher"
            else:
                direction = "lower"
                
            stds = round(abs(current_temp - pred)/1.55,2)
                
            
            lbl = Label(value=f'The current temperature is {stds} standard deviations {direction}')
            lbl.add_class(f'style_{filter_by}')
            display(lbl)
            lbl = Label(value=f'from the expected average monthly temperature for {current_city}.')
            lbl.add_class(f'style_{filter_by}')
            display(lbl)

        with self.o4:
            clear_output()
            sns.lineplot(x='Year',y='AverageTemperature',data=df.query('Year < 2040')).set_title('Average Yearly Temperature for '+ current_city)
            plt.show()
            months = ['January','February','March','April','May','June','July','August','September','October','November','December']
            month_df = mydf.query("Year ==" +str(current_year))
            sns.lineplot(x=months,y='AverageTemperature',data=month_df).set_title('Predicted Monthly Temperature for '+ current_city +' in '+str(current_year))
            plt.xticks(rotation=45)
            plt.grid()
            plt.show()
            

        with self.o5:
            clear_output()
            display(Label(value= f'Code last run {dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}' ))


d = TemperatureApp()
            
   

    

HTML(value='\n            <style>\n                /* enlarges the default jupyter cell outputs, can revert by…